In [2]:
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "0.20"

In [3]:
tf.random.set_seed(42)
np.random.seed(42)

<IPython.core.display.Javascript object>

In [4]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [6]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization_3 (Batc  (None, 784)              3136      
 hNormalization)                                                 
                                                                 
 dense_3 (Dense)             (None, 100)               78500     
                                                                 
 batch_normalization_4 (Batc  (None, 100)              400       
 hNormalization)                                                 
                                                                 
 dense_4 (Dense)             (None, 30)                3030      
                                                                 
 batch_normalization_5 (Batc  (None, 30)              

In [7]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization_3/gamma:0', True),
 ('batch_normalization_3/beta:0', True),
 ('batch_normalization_3/moving_mean:0', False),
 ('batch_normalization_3/moving_variance:0', False)]

In [9]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="mse", optimizer=optimizer)

### Transfer learning with Keras

#### create my_model_A.h5

In [15]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
tf.random.set_seed(42)
np.random.seed(42)

<IPython.core.display.Javascript object>

In [17]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [18]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [19]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 11s 7ms/step - loss: 0.5926 - accuracy: 0.8103 - val_loss: 0.3890 - val_accuracy: 0.8677
Epoch 2/20
1375/1375 [==============================] - 8s 6ms/step - loss: 0.3523 - accuracy: 0.8786 - val_loss: 0.3290 - val_accuracy: 0.8822
Epoch 3/20
1375/1375 [==============================] - 8s 6ms/step - loss: 0.3170 - accuracy: 0.8895 - val_loss: 0.3014 - val_accuracy: 0.8989
Epoch 4/20
1375/1375 [==============================] - 10s 7ms/step - loss: 0.2972 - accuracy: 0.8974 - val_loss: 0.2892 - val_accuracy: 0.9016
Epoch 5/20
1375/1375 [==============================] - 11s 8ms/step - loss: 0.2834 - accuracy: 0.9023 - val_loss: 0.2774 - val_accuracy: 0.9071
Epoch 6/20
1375/1375 [==============================] - 9s 7ms/step - loss: 0.2729 - accuracy: 0.9063 - val_loss: 0.2733 - val_accuracy: 0.9071
Epoch 7/20
1375/1375 [==============================] - 10s 7ms/step - loss: 0.2640 - accuracy: 0.9091 - val_loss: 0.2719 - val_accur

In [22]:
model_A.save("my_model_A.h5")

#### Load my_model_A.h5

In [23]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [24]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [25]:
# freeze the reused layers during the first few epochs
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False
model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd",
                     metrics=["accuracy"])

In [27]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 0s 80ms/step - loss: 0.1435 - accuracy: 0.9600 - val_loss: 0.1457 - val_accuracy: 0.9817
Epoch 2/4
7/7 [==============================] - 0s 28ms/step - loss: 0.1187 - accuracy: 0.9800 - val_loss: 0.1264 - val_accuracy: 0.9848
Epoch 3/4
7/7 [==============================] - 0s 29ms/step - loss: 0.1018 - accuracy: 0.9850 - val_loss: 0.1121 - val_accuracy: 0.9858
Epoch 4/4
7/7 [==============================] - 0s 28ms/step - loss: 0.0891 - accuracy: 0.9850 - val_loss: 0.1016 - val_accuracy: 0.9858


In [28]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

In [29]:
optimizer = keras.optimizers.SGD(lr=1e-4)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])

/Users/jinlingxing/opt/anaconda3/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [30]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_valid_B, y_valid_B))

Epoch 1/16
7/7 [==============================] - 2s 78ms/step - loss: 0.0825 - accuracy: 0.9850 - val_loss: 0.1009 - val_accuracy: 0.9858
Epoch 2/16
7/7 [==============================] - 0s 35ms/step - loss: 0.0818 - accuracy: 0.9850 - val_loss: 0.1003 - val_accuracy: 0.9858
Epoch 3/16
7/7 [==============================] - 0s 32ms/step - loss: 0.0812 - accuracy: 0.9850 - val_loss: 0.0996 - val_accuracy: 0.9858
Epoch 4/16
7/7 [==============================] - 0s 30ms/step - loss: 0.0805 - accuracy: 0.9850 - val_loss: 0.0990 - val_accuracy: 0.9858
Epoch 5/16
7/7 [==============================] - 0s 29ms/step - loss: 0.0799 - accuracy: 0.9850 - val_loss: 0.0984 - val_accuracy: 0.9858
Epoch 6/16
7/7 [==============================] - 0s 38ms/step - loss: 0.0792 - accuracy: 0.9850 - val_loss: 0.0978 - val_accuracy: 0.9858
Epoch 7/16
7/7 [==============================] - 0s 38ms/step - loss: 0.0786 - accuracy: 0.9850 - val_loss: 0.0972 - val_accuracy: 0.9858
Epoch 8/16
7/7 [===========

In [31]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 4ms/step - loss: 0.0877 - accuracy: 0.9915


[0.08773497492074966, 0.9915000200271606]

### L1 & L2 regularization

In [2]:
from functools import partial
RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(100),
    RegularizedDense(30),
    RegularizedDense(10, activation="softmax", kernel_initializer="glorot_uniform")
])


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>